# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

#### Created by Silvia Huang, CFFS Data Analyst
***

## Part IV: Data Analysis

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
pd.set_option('mode.chained_assignment', None)

/Users/jennylee/opt/anaconda3/lib/python3.9/site-packages/pdpipe/col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Set the root path, change the the current working directory into the project folder
path = os.getcwd()
os.chdir(path)

In [4]:
# Enable reading data table in the scrolling window if you prefer
#pd.set_option("display.max_rows", None, "display.max_columns", None)

***
## Import Cleaned Datasets

In [5]:
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [6]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3617,ALLSPICE GROUND SHAKER,8.0,each,454.000,g,SPICES
1,I-4472,AVOCADO MX,20.0,CT,1.000,CT,PRODUCE
2,I-4973,AVOCADO PULP CHUNKY,6.0,each,907.185,g,PRODUCE
3,I-65741,BACON 2.5MM NATURALLY SMKD,5.0,Kg,1.000,Kg,MEAT
4,I-27410,BACON 3MM NATURALLY SMKD,5.0,Kg,1.000,Kg,MEAT


In [7]:
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [8]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.00,Kg,1.00000,1.0000,P-10113
1,I-4626,16.00,CT,1.00000,2.6667,P-10115
2,I-3388,1.00,L,1.00000,0.3058,P-10496
3,I-4660,2.27,Kg,2.20462,0.6942,P-10496
4,I-12176,4.00,g,0.00100,0.3636,P-12112


In [9]:
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Preps_List_Cleaned.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
StdQty            float64
StdUom             object
dtype: object

In [10]:
Preps.head()
Preps.shape

(742, 7)

In [11]:
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [12]:
Products.head()

,ProdId,Description,SalesGroup
0,R-44765,ADD|Gravy|3oz,GV GRILL DAY
1,R-64698,FF|Burger|TMRW mushroom crunch,GV FRESH FARE DAY
2,R-61978,FLX|Fish Fritters|160g| AAD,GV - FLEX BC
3,R-56157,GRL|Big Burger|kimchiBacon|AAD,GV GRILL FEATURE
4,R-44655,GRL|Big Burger|Lumberjack|AAD,GV GRILL DAY


In [13]:
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [14]:
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L
...,...,...,...,...,...,...
629,I-55629,0.002817,1.0,ea,355.00,g
630,I-55627,0.002817,1.0,each,355.00,g
631,I-52119,0.002817,1.0,ea,355.00,g
632,I-52121,0.002817,1.0,ea,355.00,g


In [15]:
mapping = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Mapping.csv"))
mapping.dtypes

ItemId                                                     object
CategoryID                                                  int64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
g N lost/kg product                                       float64
Freshwater Withdrawals (L/FU)                             float64
Stress-Weighted Water Use (L/FU)                          float64
dtype: object

In [16]:
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,41.3463,329.5,1677.2,61309.0
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,41.3463,329.5,1677.2,61309.0
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,41.3463,329.5,1677.2,61309.0
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,41.3463,329.5,1677.2,61309.0
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,41.3463,329.5,1677.2,61309.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2036,I-55631,55,TONICA ORIGINAL 355ML,12.00,each,1.0,each,BEVERAGE,0.0000,0.0,1.0,1.0
2037,I-52119,55,WILD TEA Kombucha Cola,24.00,ea,1.0,ea,BEVERAGE,0.0000,0.0,1.0,1.0
2038,I-52121,55,WILD TEA Kombucha Cream Soda,24.00,ea,1.0,ea,BEVERAGE,0.0000,0.0,1.0,1.0
2039,I-52118,55,WILD TEA Kombucha Orange,24.00,ea,1.0,ea,BEVERAGE,0.0000,0.0,1.0,1.0


In [17]:
check = mapping["ItemId"].duplicated().any()
check

False

In [18]:
mapping.loc[mapping["ItemId"] == 'I-62863']

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
1988,I-62863,59,CK|G&G|HMR|Meat Ball & Pasta.,1.0,ea,1.0,ea,PRODUCTION FOOD,0.0,0.0,0.0,0.0


### Unit Converter

In [19]:
# Import standard unit conversion information for items
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [20]:
# Import list of prep that need convert uom to standard uom manually
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))
Manual_PrepU.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,1620.0,g
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g


In [21]:
# Add unit conversion info for preps into converter
Prep_cov = Manual_PrepU[['PrepId', 'PakQty','PakUOM','StdQty','StdUom']]
Prep_cov.insert(1, "Multiplier", '')
Prep_cov.columns = Conversions.columns
Prep_cov.loc['Multiplier'] = Prep_cov['ConvertFromQty']/Prep_cov['ConvertToQty']
Prep_cov.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,P-54697,,8.0,each,84.0,g
1,P-35132,,185.0,ea,24050.0,g
2,P-51992,,36.0,slice,1620.0,g
3,P-26234,,16.0,ea,1280.0,g
4,P-26170,,1.0,ea,125.0,g


In [22]:
frames = [Conversions, Prep_cov]
Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.0,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.5,1.0,2L,2.00,L
...,...,...,...,...,...,...
847,P-50511,,1.0,each,65.00,g
848,P-32729,,4.0,each,340.00,g
849,P-65742,,1.0,each,81.00,g
850,P-56612,,2.0,ea,142.00,g


In [23]:
# Seperate uoms that converted to 'ml' or 'g'
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [24]:
# Construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [25]:
# Test the std_converter
std_converter(0.25,'lb')

(113.398, 'g')

In [26]:
# Construct a unit converter for specific items
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit: #convert to std uom for ingredients has no specific convention instruction
        return std_converter(qty, uom)
    elif ingre in spc_cov: #convert to std uom for ingredients has specific convention instruction
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        conversion.drop_duplicates(subset=['ConversionId'], inplace = True)
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            #print(conversion)
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [27]:
spc_cov

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'I-1813',
 'I-1813',
 'I-1820',
 'I-1820',
 'I-2208',
 'I-2208',
 'I-2254',
 'I-2254',
 'I-2866',
 'I-3141',
 'I-3144',
 'I-3144',
 'I-3144',
 'I-3149',
 'I-3159',
 'I-3321',
 'I-3348',
 'I-3370',
 'I-3386',
 'I-3386',
 'I-3386',
 'I-3388',
 'I-3388',
 'I-3388',
 'I-3392',
 'I-3392',
 'I-3392',
 'I-3449',
 'I-3575',
 'I-3577',
 'I-3577',
 'I-3590',
 'I-3617',
 'I-3617',
 'I-3619',
 'I-3619',
 'I-3622',
 'I-3622',
 'I-3623',
 'I-3623',
 'I-3624',
 'I-3624',
 'I-3625',
 'I-3625',
 'I-3630',
 'I-3632',
 'I-3632',
 'I-3634',
 'I-3634',
 'I-3638',
 'I-3638',
 'I-3639',
 'I-3639',
 'I-3642',
 'I-3643',
 'I-3644',
 'I-3649',
 'I-3658',
 'I-3658',
 'I-3660',
 'I-3660',
 'I-3692',
 'I-3692',
 'I-3692',
 'I-3692',
 'I-3695',
 'I-3695',
 'I-3695',
 'I-3698',
 'I-3698',
 'I-3698

In [28]:
# Test the spc_converter
#spc_converter('I-1120', 1, 'CT')

In [29]:
spc_converter('P-35132', 1, 'ea')

(129.9999948000002, 'g')

## GHG Factors Calculation for Preps

In [30]:
Preps['GHG Emission (g)'] = 0
Preps['GHG Emission(g)/StdUom'] = 0
Preps['N lost (g)'] = 0
Preps['N lost (g)/StdUom'] = 0
Preps['Freshwater Withdrawals (ml)'] = 0
Preps['Freshwater Withdrawals (ml)/StdUom'] = 0
Preps['Stress-Weighted Water Use (ml)'] = 0
Preps['Stress-Weighted Water Use (ml)/StdUom'] = 0

In [31]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.00,Kg,1.00000,1.0000,P-10113
1,I-4626,16.00,CT,1.00000,2.6667,P-10115
2,I-3388,1.00,L,1.00000,0.3058,P-10496
3,I-4660,2.27,Kg,2.20462,0.6942,P-10496
4,I-12176,4.00,g,0.00100,0.3636,P-12112


In [32]:
Preps.loc[Preps["PrepId"] == "P-67984"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission (g),GHG Emission(g)/StdUom,N lost (g),N lost (g)/StdUom,Freshwater Withdrawals (ml),Freshwater Withdrawals (ml)/StdUom,Stress-Weighted Water Use (ml),Stress-Weighted Water Use (ml)/StdUom


In [33]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission (g),GHG Emission(g)/StdUom,N lost (g),N lost (g)/StdUom,Freshwater Withdrawals (ml),Freshwater Withdrawals (ml)/StdUom,Stress-Weighted Water Use (ml),Stress-Weighted Water Use (ml)/StdUom
0,P-27317,BATCH|3 Bean Vegetable Chili,21.00,Kg,PREP,21000.0,g,0,0,0,0,0,0,0,0
1,P-56699,BATCH|Cauliflower Wings|Baked,1.00,Kg,NaN,1000.0,g,0,0,0,0,0,0,0,0
2,P-64163,Batch|Chicken Burger|Shoyu,7.65,Kg,PREP,7650.0,g,0,0,0,0,0,0,0,0
3,P-26333,BATCH|Citrus Herb Aioli,3.00,L,PREP,3000.0,ml,0,0,0,0,0,0,0,0
4,P-53598,BATCH|Coleslaw mix,1.50,Kg,NaN,1500.0,g,0,0,0,0,0,0,0,0


In [34]:
# Calculate GHG, nitro, water footprints per gram/ml of each prep for items as ingredients only
def get_items_ghge_prep(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    nitro = Preps.loc[index, 'N lost (g)']
    water = Preps.loc[index, 'Freshwater Withdrawals (ml)']
    str_water = Preps.loc[index, 'Stress-Weighted Water Use (ml)']
    weight = Preps.loc[index, 'StdQty']
    #print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            print(ingre)
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            print(ghge)
            nitro_fac = mapping.loc[mapping['ItemId'] == ingre, 'g N lost/kg product']
            water_fac = mapping.loc[mapping['ItemId'] == ingre, 'Freshwater Withdrawals (L/FU)']
            str_water_fac = mapping.loc[mapping['ItemId'] == ingre, 'Stress-Weighted Water Use (L/FU)']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                print(ghge)
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)/1000
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            else:
                print("Test")
                qty = std_converter(Qty, Uom)[0]
                print(ghge)
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)/1000
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            #print(ingre, Qty, Uom, qty, float(ghge), qty*float(ghge))
            #print(ghg, nitro, water, str_water)
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)
    Preps.loc[index, 'N lost (g)'] = float(nitro)
    Preps.loc[index, 'N lost (g)/StdUom'] = nitro/float(weight)
    Preps.loc[index, 'Freshwater Withdrawals (ml)'] = float(water)
    Preps.loc[index, 'Freshwater Withdrawals (ml)/StdUom'] = water/float(weight)
    Preps.loc[index, 'Stress-Weighted Water Use (ml)'] = float(str_water)
    Preps.loc[index, 'Stress-Weighted Water Use (ml)/StdUom'] = str_water/float(weight)

In [35]:
for index, row in Preps.iterrows():
    get_items_ghge_prep(index , row)

I-2866
302    1.6776
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
302    1.6776
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3144
288    1.6776
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
288    1.6776
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3149
281    1.6776
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
281    1.6776
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3388
1164    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1164    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3622
1238    9.3703
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1238    9.3703
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3630
673    4.2854
Name: Active Total Supply Chai

Test
1221    9.3703
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3370
1497    2.326
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1497    2.326
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3388
1164    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1164    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3692
1408    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1408    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-5983
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-60772
1167    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1167    3.2401
Name: Active Total Supply Chain

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4657
940    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
940    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4894
927    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
927    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-5983
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-6026
1397    0.44
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1397    0.44
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-32246
914    0.3062
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
914    0.3062
Name: Active Total Supply Chain Emissions (kg

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4787
969    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
969    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-27410
42    9.8315
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
42    9.8315
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3448
974    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
974    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-13050
902    0.3062
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
902    0.3062
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4521
903    0.3062
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
903    0.3062
Name: Active Total Supply Chai

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-5983
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-1813
811    0.3942
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
811    0.3942
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3284
1504    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1504    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3660
1139    1.6414
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1139    1.6414
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4907
792    1.6547
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
792    1.6547
Name: Active Total Supply Chain Emissions (kg CO2 / kg f

...

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4538
849    0.622
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
849    0.622
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3388
1164    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1164    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-47774
1424    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1424    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4796
963    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
963    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-6026
1397    0.44
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1397    0.44
Name: Active Total Supply Chain Emission

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-43722
1494    0.70168
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1494    0.70168
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-5983
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-60772
1167    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1167    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-12176
1396    0.44
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1396    0.44
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-21727
1415    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1415    1.93
Name: Active Total Supply Chain E

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
684    4.2854
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3660
1139    1.6414
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1139    1.6414
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-5983
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1291    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-1035
392    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
392    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-1152
385    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
385    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-11664
558    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 /

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3387
1165    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1165    3.2401
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-19615
945    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
945    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-2122
111    8.9104
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
111    8.9104
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3451
973    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
973    0.5029
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-16215
1440    0.0
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1440    0.0
Name: Active Total Supply Chain Emi

Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3695
1405    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1405    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-4978
348    0.9734
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
348    0.9734
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3621
1241    9.3703
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
1241    9.3703
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3676
884    0.6932
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
884    0.6932
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
I-3697
1403    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
Test
1403    1.93
Name: Active Total Supply Chain Emissions (kg C

In [36]:
mapping.loc[mapping["ItemId"] == "I-5012"]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
720,I-5012,31,PEACH SLCD GRADE A IQF,2.0,bag,2.5,Kg,PRODUCE,0.4306,2.7,3.5,4.7


In [37]:
# Calculate GHG, nitro, water footprints per gram/ml of each prep for other preps as ingredients
def get_preps_ghge_prep(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    nitro = Preps.loc[index, 'N lost (g)']
    water = Preps.loc[index, 'Freshwater Withdrawals (ml)']
    str_water = Preps.loc[index, 'Stress-Weighted Water Use (ml)']
    weight = Preps.loc[index, 'StdQty']
    #print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P') and len(ingres) > 1:
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            nitro_fac = Preps.loc[Preps['PrepId'] == ingre, 'N lost (g)/StdUom']
            water_fac = Preps.loc[Preps['PrepId'] == ingre, 'Freshwater Withdrawals (ml)/StdUom']
            str_water_fac = Preps.loc[Preps['PrepId'] == ingre, 'Stress-Weighted Water Use (ml)/StdUom']
            #print(ghge)
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            #print(ingre, Qty, Uom, qty, qty*float(ghge))
            #print(ghg, nitro, water, str_water)
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)
    Preps.loc[index, 'N lost (g)'] = float(nitro)
    Preps.loc[index, 'N lost (g)/StdUom'] = nitro/float(weight)
    Preps.loc[index, 'Freshwater Withdrawals (ml)'] = float(water)
    Preps.loc[index, 'Freshwater Withdrawals (ml)/StdUom'] = water/float(weight)
    Preps.loc[index, 'Stress-Weighted Water Use (ml)'] = float(str_water)
    Preps.loc[index, 'Stress-Weighted Water Use (ml)/StdUom'] = str_water/float(weight)

In [38]:
Preps["Freshwater Withdrawals (ml)"].unique()

array([5.15926000e+05, 1.25760000e+05, 3.02724700e+06, 1.89250000e+04,
       0.00000000e+00, 8.87000000e+03, 5.05000000e+01, 1.54495795e+05,
       2.09775000e+06, 4.44360000e+06, 2.71600000e+04, 2.56700000e+04,
       1.01017600e+06, 1.83750601e+06, 8.13000000e+04, 2.85124143e+05,
       1.03187100e+05, 2.53429952e+05, 1.38500000e+03, 6.66540000e+04,
       1.29607800e+07, 4.67790000e+06, 1.86990000e+06, 4.94303802e+05,
       2.15456200e+04, 4.54510000e+04, 2.30892000e+05, 9.94000000e+03,
       1.62600000e+06, 3.34311133e+04, 5.59850000e+03, 2.30643408e+04,
       1.04000000e+04, 7.16902156e+05, 1.58050000e+06, 2.01034706e+06,
       9.35580000e+04, 3.16660000e+04, 2.66000000e+03, 1.84155000e+05,
       1.51890000e+04, 5.60250000e+03, 8.77800000e+02, 1.93510026e+05,
       3.44874508e+04, 1.67218432e+04, 1.90750000e+03, 6.33000000e+05,
       5.70912400e+03, 1.31619250e+04, 6.17810000e+04, 2.43900000e+04,
       3.14160050e+03, 1.00000000e+03, 5.36300000e+03, 2.74600000e+02,
      

In [39]:
# Calculate GHG, nitro, water footprints per gram/ml of each prep for linked preps
def link_preps(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    nitro = Preps.loc[index, 'N lost (g)']
    water = Preps.loc[index, 'Freshwater Withdrawals (ml)']
    str_water = Preps.loc[index, 'Stress-Weighted Water Use (ml)']
    weight = Preps.loc[index, 'StdQty']
    if len(ingres) == 1:
        ingre = ingres.iloc[0]['IngredientId']
        if ingre.startswith('P'):
            print(ingres)
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            nitro_fac = Preps.loc[Preps['PrepId'] == ingre, 'N lost (g)/StdUom']
            water_fac = Preps.loc[Preps['PrepId'] == ingre, 'Freshwater Withdrawals (ml)/StdUom']
            str_water_fac = Preps.loc[Preps['PrepId'] == ingre, 'Stress-Weighted Water Use (ml)/StdUom']
            Qty = float(ingres.iloc[0]['Qty'])
            Uom = ingres.iloc[0]['Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg = qty*float(ghge)
#                 ghg = qty*ghge
                nitro = qty*float(nitro_fac)
                water = qty*float(water_fac)
                str_water = qty*float(str_water_fac)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg = qty*float(ghge)
                nitro = qty*float(nitro_fac)
                water = qty*float(water_fac)
                str_water = qty*float(str_water_fac)
            #print(ingre, ghge, Qty, Uom, qty, weight)
            #print(ghg, nitro, water, str_water)
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)
    Preps.loc[index, 'N lost (g)'] = float(nitro)
    Preps.loc[index, 'N lost (g)/StdUom'] = nitro/float(weight)
    Preps.loc[index, 'Freshwater Withdrawals (ml)'] = float(water)
    Preps.loc[index, 'Freshwater Withdrawals (ml)/StdUom'] = water/float(weight)
    Preps.loc[index, 'Stress-Weighted Water Use (ml)'] = float(str_water)
    Preps.loc[index, 'Stress-Weighted Water Use (ml)/StdUom'] = str_water/float(weight)

In [40]:
for index, row in Preps.iterrows():
    link_preps(index, row)

   IngredientId    Qty Uom  Conversion  InvFactor   Recipe
29      P-26514  500.0   g         1.0     1000.0  P-24750
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
319      P-34084  20.0   g       0.001       10.0  P-51508
    IngredientId    Qty Uom  Conversion  InvFactor   Recipe
395      P-51130  145.0   g         1.0     1.0357  P-56717
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
426      P-50862  50.0   g       0.001        1.0  P-64696
    IngredientId  Qty Uom  Conversion  InvFactor  Recipe
452      P-48933  1.0  Kg         1.0        1.0  P-9741
    IngredientId  Qty Uom  Conversion  InvFactor  Recipe
454       P-9734  1.0  lb         1.0        2.0  P-9768
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
205      P-19175  11.5  Kg         1.0     1.0445  P-43440
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
403      P-19175  11.5  Kg         1.0     1.0455  P-57499
    IngredientId  Qty Uom  Conversion  InvFactor  Recipe
453  

In [41]:
for index, row in Preps.iterrows():
    get_preps_ghge_prep(index, row)

In [42]:
Preps.tail(30)

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission (g),GHG Emission(g)/StdUom,N lost (g),N lost (g)/StdUom,Freshwater Withdrawals (ml),Freshwater Withdrawals (ml)/StdUom,Stress-Weighted Water Use (ml),Stress-Weighted Water Use (ml)/StdUom
712,P-61841,PREP|Tater tots,1.00,Kg,NaN,1000.000,g,530.870600,0.530871,6.463500,0.006463,3.403800e+03,3.403800,1.014476e+05,101.447600
713,P-49759,ROASTED|Squash & Tofu,2.50,Kg,NaN,2500.000,g,2409.167388,0.963667,10.145597,0.004058,1.239760e+04,4.959041,3.640295e+04,14.561180
714,P-55107,RUB|Taco|Rub,100.00,g,NaN,100.000,g,1014.632400,10.146324,0.729000,0.007290,2.689200e+03,26.892000,2.379240e+04,237.924000
715,P-44647,SALAD|Arugula & Vinaigrette,1.00,Kg,NaN,1000.000,g,825.393529,0.825394,7.931679,0.007932,5.477230e+04,54.772300,2.490227e+06,2490.226814
716,P-53614,SALSA|Corn Salsa,3.30,Kg,NaN,3300.000,g,3031.033378,0.918495,22.493805,0.006816,1.759259e+05,53.310890,4.510236e+06,1366.738130
717,P-41548,SALSA|Salsa Roja,30.00,L,PREP,30000.000,ml,13264.259880,0.442142,151.090262,0.005036,1.259496e+06,41.983192,7.124155e+07,2374.718483
718,P-7136,SAUCE|Avocado Crema,2.00,L,PREP,2000.000,ml,2817.725125,1.408863,14.129069,0.007065,6.208032e+04,31.040160,1.005657e+07,5028.282831
719,P-44630,SAUCE|Chili Chilean Pebre,1.00,L,PREP,1000.000,ml,1211.363405,1.211363,7.536711,0.007537,8.120034e+04,81.200335,4.837998e+06,4837.998310
720,P-35898,SAUCE|green tobasco|Creme,7.50,L,PREP,7500.000,ml,33949.220000,4.526563,88.010000,0.011735,1.016700e+05,13.556000,9.215002e+07,12286.669867
721,P-57070,SAUCE|Pozole sauce|CAL,0.50,L,NaN,500.000,ml,3377.597778,6.755196,6.451719,0.012903,1.148872e+04,22.977441,3.594964e+05,718.992780


In [43]:
Ingredients.loc[Ingredients["IngredientId"] == "P-50509"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [44]:
Preps["GHG Emission (g)"].unique()

array([2.75343074e+04, 1.45498464e+03, 3.68115346e+04, 1.16364881e+04,
       9.93531067e+02, 2.02405458e+03, 7.18444554e+03, 1.70195520e+03,
       7.61250000e+03, 5.47096144e+04, 2.24980000e+02, 5.59009000e+03,
       3.44063113e+04, 1.14316000e+04, 2.13735695e+04, 3.62904983e+02,
       5.02900000e+02, 7.03303503e+03, 5.60395500e+02, 6.25171049e+03,
       9.75837250e+04, 8.12730592e+02, 8.20211143e+02, 1.55192213e+05,
       3.43631156e+02, 2.67312000e+04, 1.15667000e+04, 3.05763078e+03,
       3.80842086e+01, 2.80648889e+02, 3.41894970e+03, 4.91916094e+03,
       6.05903614e+02, 7.14639175e+02, 3.86925000e+03, 5.59177160e+03,
       1.42823600e+03, 1.22290400e+03, 1.00580000e+04, 1.07705376e+04,
       1.19653309e+04, 1.42635373e+04, 2.52862500e+03, 2.25879744e+03,
       8.40542500e+03, 1.76678660e+04, 5.34624000e+02, 2.33756759e+02,
       6.31689067e+03, 1.64899472e+03, 5.71588300e+03, 2.54831750e+03,
       1.89794154e+03, 1.19221768e+04, 3.60495695e+03, 6.29038345e+03,
      

In [45]:
path = os.path.join(os.getcwd(), "data", "final", "Preps Footprints.csv")
Preps.to_csv(path, index = False, header = True)

## GHGe Calculation for Products

In [46]:
Products['Weight (g)'] = 0
Products['GHG Emission (g)'] = 0
Products['N lost (g)'] = 0
Products['Freshwater Withdrawals (ml)'] = 0
Products['Stress-Weighted Water Use (ml)'] = 0

In [47]:
# Calculate GHG, nitro, water footprints per gram/ml of each product for items ingredients only
def get_items_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    nitro = Products.loc[index, 'N lost (g)']
    water = Products.loc[index, 'Freshwater Withdrawals (ml)']
    str_water = Products.loc[index, 'Stress-Weighted Water Use (ml)']
    weight = Products.loc[index, 'Weight (g)']
    #print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            nitro_fac = mapping.loc[mapping['ItemId'] == ingre, 'g N lost/kg product']
            water_fac = mapping.loc[mapping['ItemId'] == ingre, 'Freshwater Withdrawals (L/FU)']
            str_water_fac = mapping.loc[mapping['ItemId'] == ingre, 'Stress-Weighted Water Use (L/FU)']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)/1000
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            else:
                qty = std_converter(Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)/1000
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            #print(ingre, Qty, Uom, qty, float(ghge), qty*float(ghge))
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)
    Products.loc[index, 'N lost (g)'] = float(nitro)
    Products.loc[index, 'Freshwater Withdrawals (ml)'] = float(water)
    Products.loc[index, 'Stress-Weighted Water Use (ml)'] = float(str_water)

In [48]:
# Calculate GHG, nitro, water footprints per gram/ml of each product for preps ingredients only
def get_preps_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    nitro = Products.loc[index, 'N lost (g)']
    water = Products.loc[index, 'Freshwater Withdrawals (ml)']
    str_water = Products.loc[index, 'Stress-Weighted Water Use (ml)']
    weight = Products.loc[index, 'Weight (g)']
    #print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P'):
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            nitro_fac = Preps.loc[Preps['PrepId'] == ingre, 'N lost (g)/StdUom']
            water_fac = Preps.loc[Preps['PrepId'] == ingre, 'Freshwater Withdrawals (ml)/StdUom']
            str_water_fac = Preps.loc[Preps['PrepId'] == ingre, 'Stress-Weighted Water Use (ml)/StdUom']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            else:
                qty = std_converter(Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
                nitro += qty*float(nitro_fac)
                water += qty*float(water_fac)
                str_water += qty*float(str_water_fac)
            #print(ingre, Qty, Uom, qty, float(ghge), qty*float(ghge))
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)
    Products.loc[index, 'N lost (g)'] = float(nitro)
    Products.loc[index, 'Freshwater Withdrawals (ml)'] = float(water)
    Products.loc[index, 'Stress-Weighted Water Use (ml)'] = float(str_water)

In [49]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.00,Kg,1.00000,1.0000,P-10113
1,I-4626,16.00,CT,1.00000,2.6667,P-10115
2,I-3388,1.00,L,1.00000,0.3058,P-10496
3,I-4660,2.27,Kg,2.20462,0.6942,P-10496
4,I-12176,4.00,g,0.00100,0.3636,P-12112
...,...,...,...,...,...,...
4305,P-31957,20.00,g,0.00100,1.0000,R-69619
4306,P-46546,10.00,g,0.00100,1.0000,R-69619
4307,P-50757,35.00,g,0.00100,1.0000,R-69619
4308,P-52620,1.00,piece,1.00000,1.0000,R-69619


In [50]:
Products

,ProdId,Description,SalesGroup,Weight (g),GHG Emission (g),N lost (g),Freshwater Withdrawals (ml),Stress-Weighted Water Use (ml)
0,R-44765,ADD|Gravy|3oz,GV GRILL DAY,0,0,0,0,0
1,R-64698,FF|Burger|TMRW mushroom crunch,GV FRESH FARE DAY,0,0,0,0,0
2,R-61978,FLX|Fish Fritters|160g| AAD,GV - FLEX BC,0,0,0,0,0
3,R-56157,GRL|Big Burger|kimchiBacon|AAD,GV GRILL FEATURE,0,0,0,0,0
4,R-44655,GRL|Big Burger|Lumberjack|AAD,GV GRILL DAY,0,0,0,0,0
...,...,...,...,...,...,...,...,...
298,R-65512,GZA|Bowl|Spicy albacore Tuna,GV GYAZA,0,0,0,0,0
299,R-50361,GZA|Bowl|Spicy Beef|AAD,GV GYAZA,0,0,0,0,0
300,R-51249,GZA|Dashi+Dumps|AAD,GV GYAZA,0,0,0,0,0
301,R-68982,GZA|ramen broth|Udon,SPECIALS,0,0,0,0,0


In [51]:
Products["ProdId"].unique()

array(['R-44765', 'R-64698', 'R-61978', 'R-56157', 'R-44655', 'R-68378',
       'R-36098', 'R-65248', 'R-69135', 'R-53579', 'R-61830', 'R-44931',
       'R-61982', 'R-55469', 'R-28251', 'R-56454', 'R-45150', 'R-43964',
       'R-52133', 'R-50669', 'R-50668', 'R-68406', 'R-36139', 'R-53419',
       'R-44775', 'R-57094', 'R-56456', 'R-50491', 'R-56493', 'R-44770',
       'R-45233', 'R-44774', 'R-69609', 'R-69629', 'R-64866', 'R-61859',
       'R-51215', 'R-36647', 'R-47287', 'R-65755', 'R-62001', 'R-68110',
       'R-48357', 'R-62379', 'R-54377', 'R-50312', 'R-62002', 'R-68109',
       'R-62012', 'R-44024', 'R-48488', 'R-44650', 'R-68107', 'R-61844',
       'R-51146', 'R-67745', 'R-50569', 'R-50570', 'R-69126', 'R-68973',
       'R-51383', 'R-68108', 'R-36646', 'R-68099', 'R-61822', 'R-44738',
       'R-62249', 'R-54846', 'R-54202', 'R-64951', 'R-27275', 'R-65942',
       'R-43738', 'R-57179', 'R-54039', 'R-55057', 'R-64398', 'R-65761',
       'R-35341', 'R-50333', 'R-65513', 'R-40621', 

In [52]:
# Calculate GHG, nitro, water footprints per gram/ml of each product for other products ingredients
def get_products_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    nitro = Products.loc[index, 'N lost (g)']
    water = Products.loc[index, 'Freshwater Withdrawals (ml)']
    str_water = Products.loc[index, 'Stress-Weighted Water Use (ml)']
    weight = Products.loc[index, 'Weight (g)']
    #print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('R'):
            ghge = Products.loc[Products['ProdId'] == ingre, 'GHG Emission (g)']
            nitro_fac = Products.loc[Products['ProdId'] == ingre, 'N lost (g)']
            water_fac = Products.loc[Products['ProdId'] == ingre, 'Freshwater Withdrawals (ml)']
            str_water_fac = Products.loc[Products['ProdId'] == ingre, 'Stress-Weighted Water Use (ml)']
            Weight = Products.loc[Products['ProdId'] == ingre, 'Weight (g)']
            Qty = float(ingres.loc[idx,'Qty'])
            ghg += Qty*float(ghge)
            nitro += Qty*float(nitro_fac)
            water += Qty*float(water_fac)
            str_water += Qty*float(str_water_fac)
            weight += Qty*float(Weight)
            #print(ingre, Qty, float(ghge), Qty*float(ghge))
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)
    Products.loc[index, 'N lost (g)'] = float(nitro)
    Products.loc[index, 'Freshwater Withdrawals (ml)'] = float(water)
    Products.loc[index, 'Stress-Weighted Water Use (ml)'] = float(str_water)

In [53]:
for index, row in Products.iterrows():
    get_items_ghge(index , row)

In [54]:
for index, row in Products.iterrows():
    get_preps_ghge(index, row)

ValueError: could not convert string to float: ''

In [ ]:
for index, row in Products.iterrows():
    get_products_ghge(index, row)

In [ ]:
# Filter out products using preps with unknown units
Preps_Nonstd = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv"))
Preps_Nonstd

In [ ]:
def filter_products(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    #print(ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre in Preps_Nonstd['PrepId'].tolist():
            print(ingre, index, Products.loc[index,'ProdId'])
            Products.drop(index, inplace=True)
            break

In [ ]:
for index, row in Products.iterrows():
    filter_products(index, row)

In [ ]:
Products['Freshwater Withdrawals (L)'] = round(Products['Freshwater Withdrawals (ml)']/1000, 2)
Products['Stress-Weighted Water Use (L)'] = round(Products['Stress-Weighted Water Use (ml)']/1000, 2)
Products = Products.drop(columns=['Freshwater Withdrawals (ml)', 'Stress-Weighted Water Use (ml)'])

In [ ]:
Products['GHG Emission (g) / 100g'] = round(100*Products['GHG Emission (g)']/Products['Weight (g)'], 2)
Products['N lost (g) / 100g'] = round(100*Products['N lost (g)']/Products['Weight (g)'], 2)
Products['Freshwater Withdrawals (L) / 100g'] = round(100*Products['Freshwater Withdrawals (L)']/Products['Weight (g)'], 2)
Products['Stress-Weighted Water Use (L) / 100g'] = round(100*Products['Stress-Weighted Water Use (L)']/Products['Weight (g)'], 2)

In [ ]:
Products

In [ ]:
Products.shape

In [ ]:
path = os.path.join(os.getcwd(), "data", "final", "Recipes Footprints.csv")
Products.to_csv(path, index = False, header = True)

## Data Visualization

In [ ]:
path = os.path.join(os.getcwd(), "reports", "figures/")

In [ ]:
Products.boxplot(column=['GHG Emission (g)'], return_type='axes')

In [ ]:
Products.boxplot(column=['N lost (g)'], return_type='axes')

In [ ]:
Products.boxplot(column=['Freshwater Withdrawals (L)'], return_type='axes')

In [ ]:
Products.boxplot(column=['Stress-Weighted Water Use (L)'], return_type='axes')

In [ ]:
Products.hist(column=['GHG Emission (g)'], bins= 40, alpha = 0.7)
plt.axvline(Products['GHG Emission (g)'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['GHG Emission (g)'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('GHG Emission (g) / Dish')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'GHGe_dish.png')

In [ ]:
Products.hist(column=['GHG Emission (g) / 100g'], bins= 40, alpha = 0.7)
plt.axvline(Products['GHG Emission (g) / 100g'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['GHG Emission (g) / 100g'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('GHG Emission (g) / 100g')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'GHGe_100g.png')

In [ ]:
Products.hist(column=['N lost (g)'], bins= 40, alpha = 0.7)
plt.axvline(Products['N lost (g)'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['N lost (g)'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('N lost (g) / Dish')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'N lost_dish.png')

In [ ]:
Products.hist(column=['N lost (g) / 100g'], bins= 40, alpha = 0.7)
plt.axvline(Products['N lost (g) / 100g'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['N lost (g) / 100g'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('N lost (g) / 100g')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'N lost_100g.png')

In [ ]:
Products.hist(column=['Freshwater Withdrawals (L)'], bins= 40, alpha = 0.7)
plt.axvline(Products['Freshwater Withdrawals (L)'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['Freshwater Withdrawals (L)'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('Freshwater Withdrawals (L) / Dish')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'Fresh water_dish.png')

In [ ]:
Products.hist(column=['Freshwater Withdrawals (L) / 100g'], bins= 40, alpha = 0.7)
plt.axvline(Products['Freshwater Withdrawals (L) / 100g'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['Freshwater Withdrawals (L) / 100g'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('Freshwater Withdrawals (L) / 100g')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'Fresh water_100g.png')

In [ ]:
Products.hist(column=['Stress-Weighted Water Use (L)'], bins= 40, alpha = 0.7)
plt.axvline(Products['Stress-Weighted Water Use (L)'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['Stress-Weighted Water Use (L)'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('Stress-Weighted Water Use (L) / Dish')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'Stress water_dish.png')

In [ ]:
Products.hist(column=['Stress-Weighted Water Use (L) / 100g'], bins= 40, alpha = 0.7)
plt.axvline(Products['Stress-Weighted Water Use (L) / 100g'].mean(), color='r', linestyle='dashed', linewidth=2, label = 'mean' )
plt.axvline(Products['Stress-Weighted Water Use (L) / 100g'].median(), color='k', linewidth=1, label = 'median')
plt.xlabel('Stress-Weighted Water Use (L) / 100g')
plt.ylabel('Frequency')
plt.xlim(left=0)
plt.savefig(path + 'Stress water_100g.png')